<a href="https://colab.research.google.com/github/jeong-sang-choi/emotional-analyis-game-/blob/maseter/FIFA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**LLM모델(genai)을 이용한 피파온라인 유저의 감성분석**

In [5]:
import pandas as pd
import numpy as np
import pathlib
import textwrap
import time
import google.generativeai as genai
from google.colab import userdata
from IPython.display import display
from IPython.display import Markdown

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [8]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [9]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.0-flash-exp
models/gemini-exp-1206
models/gemini-exp-1121
models/gemini-exp-1114
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/learnlm-1.5-pro-experimental


In [10]:
# 텍스트 전용
model = genai.GenerativeModel('gemini-1.5-flash')

In [11]:
file_path = "/content/drive/My Drive/fifa/dataset1.csv"

In [12]:
fifa_df = pd.read_csv(file_path)
fifa_df.head()

,date,community,postId,title,body,viewCount,commentCount
0,2020-07-01,인벤,16258,스몰링 사재기 갑니다.jpg,3억5천바라봅니다,227,11
1,2020-07-01,인벤,36379,현재까지 빠칭코 출시 시기 모음.,"토티, 토츠 출시 1주일 후에 전부다 빠짐없이 빠찡코 (럭키상점) 등장100퍼센트는...",192,12
2,2020-07-01,인벤,28187,좀전에 3바란뜬사람,피파하면서 비싼거 강화 절대안하는데 4강까지 갓으니 걍 해보렵니다ㅠ,116,22
3,2020-07-01,인벤,28331,주공 날강두 은카 결과,감사합니다 인벤행님들착하게 살게요 ㅋㅋ,132,11
4,2020-07-01,에펨코,1392,1277 은카 파시는 선생님,저는 일단 자보겠읍니다.댓글로 남겼지만팩트 하나 말씀드리자면오버롤 107로 개높은 ...,129,17


In [13]:
def make_review_pn(review):
  prompt = f'''
  아래 리뷰를 읽고 긍정인지 부정, 알려줘 그리고 만약 긍정이면 1을 반환 부정이면 0을 반환해줘 예를들어 "1", "0"
  리뷰:{review}
  '''
  response = model.generate_content(prompt)

  return response.text

In [14]:
# 10개의 행만 감정분석
df_review_10 = fifa_df.iloc[:10, :]
df_review_10

,date,community,postId,title,body,viewCount,commentCount
0,2020-07-01,인벤,16258,스몰링 사재기 갑니다.jpg,3억5천바라봅니다,227,11
1,2020-07-01,인벤,36379,현재까지 빠칭코 출시 시기 모음.,"토티, 토츠 출시 1주일 후에 전부다 빠짐없이 빠찡코 (럭키상점) 등장100퍼센트는...",192,12
2,2020-07-01,인벤,28187,좀전에 3바란뜬사람,피파하면서 비싼거 강화 절대안하는데 4강까지 갓으니 걍 해보렵니다ㅠ,116,22
3,2020-07-01,인벤,28331,주공 날강두 은카 결과,감사합니다 인벤행님들착하게 살게요 ㅋㅋ,132,11
4,2020-07-01,에펨코,1392,1277 은카 파시는 선생님,저는 일단 자보겠읍니다.댓글로 남겼지만팩트 하나 말씀드리자면오버롤 107로 개높은 ...,129,17
5,2020-07-01,인벤,1340,110억짜리 ZD.gif,오늘도 나지막이 읊어봅니다.에메에에에에에에에에에에에에에에에에에에에에에에에에에에에에에...,158,18
6,2020-07-01,인벤,23358,요즘 잠잠한 피벤 1대장 근황.JPG,한 때 어그로 명단 작성 파동으로 피벤에 큰 분란을 일으켰던 나XX.요즘 근황을 알...,175,15
7,2020-07-01,인벤,3759,강화이벤트 못 기다리고 강화질렀습니다,챔시즌 나왔을 때 강화로 너무 많이 해먹어서그 이후로 오랫동안 본섭에서 강화는 손도...,106,11
8,2020-07-01,인벤,2006,20토츠 호날두 5카 후기,아직 많이 사용해보진 못했는데 당연히 헤더좋고 몸싸움 좋은데풀캐미 슛파워 122 중...,107,12
9,2020-07-01,에펨코,22150,와.... 형들 에이전트 무조건 하세요,접을 마음으로 팀 다 팔고 돌렸는데.... 다시 해야겠네요,114,11


In [15]:
df_review_10.loc[:, '긍정/부정'] = df_review_10['body'].apply(make_review_pn)

<ipython-input-15-14c309769c21>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_review_10.loc[:, '긍정/부정'] = df_review_10['body'].apply(make_review_pn)


In [16]:
df_review_10['긍정/부정'].head()

# 줄바꿈 삭제
df_review_10.loc[:, '긍정/부정'] = df_review_10['긍정/부정'].apply(lambda x: x.replace("\n", ""))

In [17]:
df_review_10.head()

,date,community,postId,title,body,viewCount,commentCount,긍정/부정
0,2020-07-01,인벤,16258,스몰링 사재기 갑니다.jpg,3억5천바라봅니다,227,11,0
1,2020-07-01,인벤,36379,현재까지 빠칭코 출시 시기 모음.,"토티, 토츠 출시 1주일 후에 전부다 빠짐없이 빠찡코 (럭키상점) 등장100퍼센트는...",192,12,0
2,2020-07-01,인벤,28187,좀전에 3바란뜬사람,피파하면서 비싼거 강화 절대안하는데 4강까지 갓으니 걍 해보렵니다ㅠ,116,22,0
3,2020-07-01,인벤,28331,주공 날강두 은카 결과,감사합니다 인벤행님들착하게 살게요 ㅋㅋ,132,11,1
4,2020-07-01,에펨코,1392,1277 은카 파시는 선생님,저는 일단 자보겠읍니다.댓글로 남겼지만팩트 하나 말씀드리자면오버롤 107로 개높은 ...,129,17,0


In [18]:
#300개의 행만 뽑아서 분석
df_review_100 = fifa_df.loc[:100:]
# df_review_30

In [22]:
# 1분에 15개씩 처리
def process_reviews_in_batches(df_review_100, batch_size=15, delay=60):
    total_rows = len(df_review_100)
    for start_idx in range(0, total_rows, batch_size):
        end_idx = min(start_idx + batch_size, total_rows)
        batch = df_review_100.loc[start_idx:end_idx-1, :].copy()

        # make_review_pn 함수를 적용
        batch['긍정/부정'] = batch['body'].apply(make_review_pn)
        # 처리된 배치를 원본 데이터프레임에 업데이트
        df_review_100.loc[start_idx:end_idx-1, '긍정/부정'] = batch['긍정/부정'].values

        # 더 처리할 행이 남아있다면 지정된 지연 시간만큼 대기
        if end_idx < total_rows:
            time.sleep(delay)  # 지정된 시간(초) 동안 대기

    return df_review_100

In [23]:
# 1분에 15개씩 나눠서 처리
processed_df = process_reviews_in_batches(df_review_100)
processed_df['긍정/부정'] = processed_df['긍정/부정'].apply(lambda x : x.replace("\n",""))

<ipython-input-22-eae7849d33f0>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_review_100.loc[start_idx:end_idx-1, '긍정/부정'] = batch['긍정/부정'].values
<ipython-input-23-971ca27ffe84>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_df['긍정/부정'] = processed_df['긍정/부정'].apply(lambda x : x.replace("\n",""))


,date,community,postId,title,body,viewCount,commentCount,긍정/부정
0,2020-07-01,인벤,16258,스몰링 사재기 갑니다.jpg,3억5천바라봅니다,227,11,0
1,2020-07-01,인벤,36379,현재까지 빠칭코 출시 시기 모음.,"토티, 토츠 출시 1주일 후에 전부다 빠짐없이 빠찡코 (럭키상점) 등장100퍼센트는...",192,12,0
2,2020-07-01,인벤,28187,좀전에 3바란뜬사람,피파하면서 비싼거 강화 절대안하는데 4강까지 갓으니 걍 해보렵니다ㅠ,116,22,0
3,2020-07-01,인벤,28331,주공 날강두 은카 결과,감사합니다 인벤행님들착하게 살게요 ㅋㅋ,132,11,1
4,2020-07-01,에펨코,1392,1277 은카 파시는 선생님,저는 일단 자보겠읍니다.댓글로 남겼지만팩트 하나 말씀드리자면오버롤 107로 개높은 ...,129,17,0
...,...,...,...,...,...,...,...,...
96,2020-07-02,인벤,17047,실축) 와 이건 진짜 모르겠다,첼시랑 레스터 둘다 갑자기 삽질하네...;;누가 챔스갈지는 진짜 모르겠네 ㄷㄷ점점 ...,125,14,0
97,2020-07-02,인벤,22829,와 적당한 -꼰- 인줄알았는데,상상을 초월하시네...,206,16,0
98,2020-07-02,에펨코,25117,이런걸로 이벤트해도 되는지 모르겠는데 가즈아 ㅋㅋㅋㅋㅋㅋㅋㅋㅋ,5만코인 이상 뜨면 5명 100포 ㄱㅈㅇ 추천하고 댓글로 응원 한마디씩 ㄱㄱ,201,35,0
99,2020-07-02,에펨코,12319,박성주는 손흥민 개혹평하네,나는 ㄹㅇ 개좋게 나온시즌이라 생각하면서 쓰는데19토츠에 비해서 체감이 둔탁하고 그...,131,25,0


In [30]:
processed_df.to_csv("emotional_fifa.csv")

In [36]:
!

/content
